In [1]:
import os
import re
import spacy
import pandas as pd

# Preprocess Text

In [2]:
FILE_PATH = "../assets/NER_assets/Ancient_Words.csv"
# read csv file
df = pd.read_csv(FILE_PATH)
# Preprocess Text

In [3]:
# rename columns to fit code
df.rename(columns = {'Word':'Keyword', 'Category Types':'Label'}, inplace = True)
# If a cell is empty (NaN), Fill it with the value in its parallel "Early" column
for row in df:
    df['Quote'].fillna(df['Early Quote'], inplace=True)
    df['Word Before'].fillna(df['Early Word Before'], inplace=True)
    df['Word After'].fillna(df['Early Word After'], inplace=True)
    df['Label'].fillna(df['Early Category Type'], inplace=True)
# remove rows with no Keyword
df = df.dropna(subset=['Keyword'])
# Remove any row that isn't Greek
pat = '[ء-ي]+'
#df.Keyword.str.contains(pat)
df = df[~df.Keyword.str.contains(pat, na=False)]
#replace new line in df column
df['Keyword'].replace('\n', '', regex=True, inplace=True)
#replace numbers in df
df.replace('\d+', '', regex=True, inplace=True)
#replace hyphens in df column
df.replace('-', '', regex=True, inplace=True)
# replace comma in df column
df['Keyword'].replace(',', '', regex=True, inplace=True)
#replace period in df column
df['Keyword'].replace('\.', '', regex=True, inplace=True)
#replace interpunkt in df column
df['Keyword'].replace('\·', '', regex=True, inplace=True)
# replace multiple spaces in df column
df.replace(' +', ' ', regex=True, inplace=True)
# replace end punctuation in df column
df['Keyword'].replace('\s+$', '', regex=True, inplace=True)

df.fillna(0)
df.reset_index(drop=True, inplace=True)


In [4]:
df.head(10)

,Keyword,Word Before,Word After,Quote,Label,Lemma,Early Category Type,Early Word,Early Word Before,Early Word After,Early Quote,Lemma arabic
0,οὖλον,δὲ πολυφυὲς,· σάρκινα δὲ,Καὶ τὸ μὲν διφυὲς τοῦ στόματος παρίσθμι...,Body Part,οὖλον,Body Part,οὖλον,δὲ πολυφυὲς,· σάρκινα δὲ,Καὶ τὸ μὲν διφυὲς τοῦ στόματος παρίσθμι...,NaN
1,παρίσθμιον,τοῦ στόματος,", τὸ δὲ",Καὶ τὸ μὲν διφυὲς τοῦ στόματος παρίσθμι...,Body Part,παρίσθμιον,Body Part,παρίσθμιον,τοῦ στόματος,", τὸ δὲ",Καὶ τὸ μὲν διφυὲς τοῦ στόματος παρίσθμι...,NaN
2,πολυφυὲς,τὸ δὲ,οὖλον· σάρκινα,Καὶ τὸ μὲν διφυὲς τοῦ στόματος παρίσθμι...,Adjectives/Qualities,πολυφυής,Adjectives/Qualities,πολυφυὲς,τὸ δὲ,οὖλον· σάρκινα,Καὶ τὸ μὲν διφυὲς τοῦ στόματος παρίσθμι...,NaN
3,μόριον,δ’ ἄλλο,"σταφυλοφόρον, κίων","Εἴσω δ’ ἄλλο μόριον σταφυλοφόρον , κίων...",Body Part,μόριον,Body Part,μόριον,δ’ ἄλλο,"σταφυλοφόρον, κίων","Εἴσω δ’ ἄλλο μόριον σταφυλοφόρον , κίων...",NaN
4,ὀδόντες,Ἐντὸς δ’,ὀστέινοι. Εἴσω,Ἐντὸς δ’ ὀδόντες ὀστέινοι,Body Part,ὀδούς,Body Part,ὀδόντες,Ἐντὸς δ’,ὀστέινοι. Εἴσω,Ἐντὸς δ’ ὀδόντες ὀστέινοι,NaN
5,ὀστέινοι,δ’ ὀδόντες,. Εἴσω δ’,Ἐντὸς δ’ ὀδόντες ὀστέινοι,Adjectives/Qualities,ὀστέινος,Adjectives/Qualities,ὀστέινοι,δ’ ὀδόντες,. Εἴσω δ’,Ἐντὸς δ’ ὀδόντες ὀστέινοι,NaN
6,σταφυλοφόρον,ἄλλο μόριον,", κίων ἐπίφλεβος·","Εἴσω δ’ ἄλλο μόριον σταφυλοφόρον , κίων...",Adjectives/Qualities,σταφυλοφόρος,Adjectives/Qualities,σταφυλοφόρον,ἄλλο μόριον,", κίων ἐπίφλεβος·","Εἴσω δ’ ἄλλο μόριον σταφυλοφόρον , κίων...",NaN
7,ἐπίφλεβος,"σταφυλοφόρον, κίων",· ὃς ἐὰν,"Εἴσω δ’ ἄλλο μόριον σταφυλοφόρον, κίων ...",Adjectives/Qualities,ἐπίφλεβος,Adjectives/Qualities,ἐπίφλεβος,"σταφυλοφόρον, κίων",· ὃς ἐὰν,"Εἴσω δ’ ἄλλο μόριον σταφυλοφόρον, κίων ...",NaN
8,κίων,"μόριον σταφυλοφόρον,",ἐπίφλεβος· ὃς,"Εἴσω δ’ ἄλλο μόριον σταφυλοφόρον, κίων ...",Adjectives/Qualities,κίων,Adjectives/Qualities,κίων,"μόριον σταφυλοφόρον,",ἐπίφλεβος· ὃς,"Εἴσω δ’ ἄλλο μόριον σταφυλοφόρον, κίων ...",NaN
9,μέρος,Ἔτι προσώπου,τὸ μὲν,Ἔτι προσώπου μέρος τὸ μὲν ὂν τῷ πνευ...,Adjectives/Qualities,μέρος,Adjectives/Qualities,μέρος,Ἔτι προσώπου,τὸ μὲν,Ἔτι προσώπου μέρος τὸ μὲν ὂν τῷ πνευ...,NaN


In [5]:
# if any of the fields "KeyWord", "Quote", "Word Before", "Word After" are "0", drop the row
for w in ['Keyword', 'Quote', 'Word Before', 'Word After']:
    df = df[df[w] != 0]


In [6]:
# import requirements for converting the dataframe to Spacy Docs
from collections import defaultdict
from typing import List
from spacy.language import Language
from spacy.tokens import Doc, DocBin, Span
from spacy.util import filter_spans
from unicodedata import normalize
import regex
import random


# Convert the dataframe to Spacy Docs

In [7]:
d = {ord('\N{COMBINING ACUTE ACCENT}'):None, ord('\N{COMBINING COMMA ABOVE}'):None, ord('\N{COMBINING REVERSED COMMA ABOVE}'):None}

def find_word_index(sentence, word, word_before, word_after):
    """Find the index of a word in a sentence, it can appear multiple times but we return by word_before and word_after"""


    # find the index of the word in the sentence (return index of beginning of word)
    word_index = sentence.find(word)

    # return all chars of word before: [),.,·] (meaning none of these chars will be included) so we get only data contained within the sentence
    for char in [")", ".", "·"]:
        if word_before.find(char) != -1:
            word_before = word_before[word_before.find(char)+1:]
        if word_after.find(char) != -1:
            word_after = word_after[:word_after.find(char)]



    pattern = fr"{re.escape(word_before)}({(word)}){re.escape(word_after)}"
    # add fuzzy matching with up to 3 mistakes
    pattern = fr"(?:{pattern}){{e<=3}}"



    match = regex.search(pattern, sentence)
    try:
        print ("match:", match.span(1))
        return match.span(1)
    except:

        return None



In [8]:
# normalize table
FORMAT = 'NFD'
for col in ['Keyword', 'Quote', 'Word Before', 'Word After']:
    df[col] = df[col].apply(lambda x: normalize(FORMAT, x))

In [9]:
# using word before and word after, we want to bring our data into the following format:
# TRAIN_DATA = [ (TEXT AS A STRING, {“entities”: [(START, END, LABEL)]}) ]

TRAIN_DATA = []
train_data_df = pd.DataFrame(columns=['text', 'keyword', 'index', 'label', 'lemma'])
for index, row in df.iterrows():

    train_data_df = train_data_df.append({'text': row['Quote'], 'keyword': row['Keyword'], 'index': find_word_index(row['Quote'], row['Keyword'], row['Word Before'], row['Word After']), 'label': row['Label'], 'lemma' : row['Lemma']}, ignore_index=True)
    print ("******")

match: (68, 76)
******
match: (36, 48)
******
match: (58, 68)
******
match: (16, 24)
******
match: (10, 20)
******
match: (20, 31)
******
match: (24, 38)
******
match: (45, 57)
******
match: (39, 45)
******
match: (15, 22)
******
match: (41, 51)
******
match: (51, 58)
******
match: (5, 15)
******
match: (58, 64)
******
match: (9, 19)
******
match: (60, 65)
******
match: (24, 33)
******
match: (51, 60)
******
match: (33, 41)
******
******
match: (72, 81)
******
match: (103, 112)
******
match: (11, 24)
******
match: (41, 50)
******
match: (50, 55)
******
match: (29, 41)
******
match: (59, 67)
******
match: (92, 98)
******
match: (85, 92)
******
match: (108, 121)
******
match: (139, 144)
******
match: (148, 152)
******
match: (125, 137)
******
match: (97, 108)
******
match: (157, 166)
******
match: (178, 189)
******
match: (215, 227)
******
match: (194, 204)
******
match: (204, 210)
******
match: (237, 241)
******
match: (246, 255)
******
match: (16, 25)
******
match: (25, 30)
******
matc

/tmp/ipykernel_506/3908230776.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data_df = train_data_df.append({'text': row['Quote'], 'keyword': row['Keyword'], 'index': find_word_index(row['Quote'], row['Keyword'], row['Word Before'], row['Word After']), 'label': row['Label'], 'lemma' : row['Lemma']}, ignore_index=True)
/tmp/ipykernel_506/3908230776.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data_df = train_data_df.append({'text': row['Quote'], 'keyword': row['Keyword'], 'index': find_word_index(row['Quote'], row['Keyword'], row['Word Before'], row['Word After']), 'label': row['Label'], 'lemma' : row['Lemma']}, ignore_index=True)
/tmp/ipykernel_506/3908230776.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat 

match: (159, 165)
******
match: (39, 46)
******
match: (110, 118)
******
match: (80, 87)
******
match: (78, 86)
******
match: (91, 98)
******
match: (35, 43)
******
match: (310, 320)
******
match: (179, 189)
******
match: (34, 45)
******
match: (52, 63)
******
match: (56, 67)
******
match: (15, 22)
******
match: (97, 109)
******
match: (234, 244)
******
match: (125, 135)
******
match: (49, 59)
******
match: (32, 40)
******
match: (35, 44)
******
match: (68, 77)
******
match: (14, 25)
******
match: (142, 148)
******
match: (0, 9)
******
match: (152, 159)
******
match: (209, 216)
******
match: (291, 300)
******
match: (23, 34)
******
match: (190, 200)
******
match: (60, 68)
******
match: (86, 96)
******
match: (44, 50)
******
match: (28, 38)
******
match: (273, 281)
******
match: (70, 80)
******
match: (13, 25)
******
match: (61, 72)
******
match: (114, 124)
******
match: (109, 114)
******
match: (98, 109)
******
match: (124, 135)
******
match: (91, 98)
******
match: (16, 26)
******
matc

/tmp/ipykernel_506/3908230776.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data_df = train_data_df.append({'text': row['Quote'], 'keyword': row['Keyword'], 'index': find_word_index(row['Quote'], row['Keyword'], row['Word Before'], row['Word After']), 'label': row['Label'], 'lemma' : row['Lemma']}, ignore_index=True)
/tmp/ipykernel_506/3908230776.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data_df = train_data_df.append({'text': row['Quote'], 'keyword': row['Keyword'], 'index': find_word_index(row['Quote'], row['Keyword'], row['Word Before'], row['Word After']), 'label': row['Label'], 'lemma' : row['Lemma']}, ignore_index=True)
/tmp/ipykernel_506/3908230776.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat 

match: (79, 89)
******
match: (51, 60)
******
match: (25, 34)
******
match: (34, 41)
******
match: (19, 30)
******
match: (30, 42)
******
match: (42, 47)
******
match: (51, 58)
******
match: (58, 64)
******
match: (0, 10)
******
match: (10, 21)
******
match: (69, 80)
******
match: (84, 92)
******
match: (28, 40)
******
match: (12, 23)
******
match: (40, 45)
******
match: (0, 3)
******
match: (45, 51)
******
match: (51, 63)
******
match: (23, 32)
******
match: (32, 37)
******
match: (93, 102)
******
match: (78, 88)
******
match: (37, 45)
******
match: (56, 67)
******
match: (36, 47)
******
match: (102, 115)
******
match: (23, 31)
******
match: (47, 59)
******
match: (0, 11)
******
match: (91, 100)
******
match: (0, 4)
******
match: (81, 91)
******
match: (8, 20)
******
match: (64, 76)
******
match: (19, 30)
******
match: (20, 29)
******
match: (0, 10)
******
match: (13, 23)
******
match: (149, 157)
******
match: (144, 149)
******
match: (119, 132)
******
match: (136, 144)
******
match: 

...

/tmp/ipykernel_506/3908230776.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data_df = train_data_df.append({'text': row['Quote'], 'keyword': row['Keyword'], 'index': find_word_index(row['Quote'], row['Keyword'], row['Word Before'], row['Word After']), 'label': row['Label'], 'lemma' : row['Lemma']}, ignore_index=True)
/tmp/ipykernel_506/3908230776.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data_df = train_data_df.append({'text': row['Quote'], 'keyword': row['Keyword'], 'index': find_word_index(row['Quote'], row['Keyword'], row['Word Before'], row['Word After']), 'label': row['Label'], 'lemma' : row['Lemma']}, ignore_index=True)
/tmp/ipykernel_506/3908230776.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat 

match: (88, 101)
******
match: (0, 5)
******
match: (6, 17)
******
match: (208, 219)
******
match: (190, 199)
******
match: (44, 59)
******
match: (33, 39)
******
match: (59, 68)
******
match: (80, 90)
******
match: (68, 80)
******
match: (99, 103)
******
match: (138, 142)
******
match: (125, 138)
******
******
******
match: (108, 115)
******
match: (169, 174)
******
match: (211, 219)
******
match: (186, 196)
******
match: (174, 186)
******
match: (265, 277)
******
match: (18, 38)
******
match: (18, 38)
******
match: (9, 13)
******
match: (24, 29)
******
match: (57, 67)
******
match: (128, 132)
******
match: (122, 128)
******
match: (132, 144)
******
match: (216, 220)
******
match: (225, 237)
******
match: (207, 215)
******
match: (16, 28)
******
match: (103, 114)
******
match: (57, 63)
******
match: (63, 73)
******
******
match: (34, 41)
******
******
match: (25, 30)
******
match: (35, 43)
******
match: (74, 78)
******
match: (103, 113)
******
match: (94, 103)
******
match: (118, 126)

/tmp/ipykernel_506/3908230776.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data_df = train_data_df.append({'text': row['Quote'], 'keyword': row['Keyword'], 'index': find_word_index(row['Quote'], row['Keyword'], row['Word Before'], row['Word After']), 'label': row['Label'], 'lemma' : row['Lemma']}, ignore_index=True)
/tmp/ipykernel_506/3908230776.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data_df = train_data_df.append({'text': row['Quote'], 'keyword': row['Keyword'], 'index': find_word_index(row['Quote'], row['Keyword'], row['Word Before'], row['Word After']), 'label': row['Label'], 'lemma' : row['Lemma']}, ignore_index=True)
/tmp/ipykernel_506/3908230776.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat 

match: (78, 87)
******
******
******
match: (141, 151)
******
match: (182, 191)
******
match: (0, 5)
******
match: (32, 39)
******
match: (15, 24)
******
match: (24, 32)
******
match: (20, 26)
******
match: (0, 11)
******
******
******
match: (41, 46)
******
******
match: (82, 87)
******
match: (101, 107)
******
******
match: (71, 76)
******
match: (142, 156)
******
match: (121, 133)
******
match: (17, 23)
******
match: (9, 16)
******
match: (31, 43)
******
match: (31, 43)
******
match: (49, 56)
******
match: (101, 109)
******
match: (33, 43)
******
match: (48, 55)
******
match: (84, 92)
******
match: (60, 65)
******
match: (19, 27)
******
match: (33, 40)
******
match: (44, 52)
******
match: (35, 41)
******
match: (63, 71)
******
match: (70, 77)
******
match: (12, 20)
******
match: (41, 50)
******
******
match: (28, 35)
******
match: (50, 58)
******
match: (187, 195)
******
match: (127, 135)
******
match: (130, 141)
******
match: (11, 18)
******
match: (48, 59)
******
match: (78, 95)
*

/tmp/ipykernel_506/3908230776.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data_df = train_data_df.append({'text': row['Quote'], 'keyword': row['Keyword'], 'index': find_word_index(row['Quote'], row['Keyword'], row['Word Before'], row['Word After']), 'label': row['Label'], 'lemma' : row['Lemma']}, ignore_index=True)
/tmp/ipykernel_506/3908230776.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data_df = train_data_df.append({'text': row['Quote'], 'keyword': row['Keyword'], 'index': find_word_index(row['Quote'], row['Keyword'], row['Word Before'], row['Word After']), 'label': row['Label'], 'lemma' : row['Lemma']}, ignore_index=True)
/tmp/ipykernel_506/3908230776.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat 

Please notice the above warning about using a deprecated method, and change accordingly.\
<code>FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.</code>


In [11]:
# compare text[index] with keyword
test_data_df = pd.DataFrame(columns=['text', 'real_keyword', 'function_found'])
# count nulls in train_data_df
nulls = train_data_df.isnull().sum()
# drop nulls in train_data_df
train_data_df = train_data_df.dropna()
# iterate over all rows in train_data_df
for index, row in train_data_df.iterrows():
    test_data_df = test_data_df.append({'text': row['text'], 'real_keyword': row['keyword'], 'function_found': row['text'][row['index'][0]:row['index'][1]]}, ignore_index=True)


/tmp/ipykernel_506/523546791.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_data_df = test_data_df.append({'text': row['text'], 'real_keyword': row['keyword'], 'function_found': row['text'][row['index'][0]:row['index'][1]]}, ignore_index=True)
/tmp/ipykernel_506/523546791.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_data_df = test_data_df.append({'text': row['text'], 'real_keyword': row['keyword'], 'function_found': row['text'][row['index'][0]:row['index'][1]]}, ignore_index=True)
/tmp/ipykernel_506/523546791.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_data_df = test_data_df.append({'text': row['text'], 'real_keyword': row['keyword'], 'function_found': row['text'][row['index'][0]:row['inde

In [12]:
# print nulls in train_data_df
print ("Nulls", nulls)
# print shape of train_data_df
print ("\nShape", train_data_df.shape)


Nulls text        0
keyword     0
index      57
label       2
lemma       5
dtype: int64

Shape (1476, 5)


In [13]:
# sample test_data_df where real_keyword != function_found
test_data_df[test_data_df['real_keyword'] != test_data_df['function_found']].sample(10)

,text,real_keyword,function_found
1456,"αἱ γὰρ φλέβες μεσταί εἰσιν αἵματος, και...",ἐγκέφαλος,ἐγκέφαλος
125,Αἰσθάνεται δὲ καὶ ὧν ἡ ἄλλη σὰρξ πά...,μέρος,μέρος
595,Ἡ μὲν οὖν μεγάλη φλὲψ ἐκ τῆς μεγίστης...,ᾧ,ᾧ
18,"καὶ γὰρ ἀναπνεῖ καὶ ἐκπνεῖ ταύτῃ , κα...",ταύτῃ,ταύτῃ
809,τὸ δ’ εἰς τὸ κάτω τῆς καρδίας μέρος τῆ...,τείνει,τείνει
1124,Ὃν μὲν οὖν τρόπον ἔχουσιν αἱ φλέβες ...,φλέβες,φλέβες
1046,Αἱ δὲ σχίσεις ἑκατέρας τῆς φλεβὸς τείν...,ἑκάτερον,ἑκάτερον
236,Ἔστι δ’ ἡ μὲν ἀρτηρία χονδρώδης τὴν φυ...,ἐκ,ἐκ
841,διὸ ἀποσχαζόντων τῶν ἰατρῶν ταύτην ἀπο...,ταύτην,ταύτην
977,εἶτα εἰς ἑκάτερον τὸ ἰσχίον ἀφανίζετ...,διατεταμέναι,διατεταμέναι


# Create dictionaries from dendrosearch and conllu files (supplied by Jackobo)

In [19]:
PUNCTUATION = ['.', ")", ".", "·", "(", "[", "]", ":", ";", ",", "?", "!", "،", "_"]
# extract from df a dictionary {word: lemma}
lemma_dict = {}
for index, row in df.iterrows():
    lemma_dict[row['Keyword']] = row['Lemma']

# load dendrosearch lemma dictionary
dendrosearch_lemma_dict = {}
with open('../assets/dendrosearch_lemma_dict.txt', 'r', encoding='utf-8') as f:
    for line in f:
        line = line.split()
        # check if not punctuation
        if len(line) > 1 and line[0] not in PUNCTUATION:
            dendrosearch_lemma_dict[line[0]] = line[1]

# create dictionary from all conllu files
PATH = "../assets/Lemmatization_training_files/"
conllu_lemma_dict = {}

# iterate over all files in directory
for f in os.listdir(PATH):
    if f.endswith(".conllu"):
        # if file is a conllu file
        with open(os.path.join(PATH, f), 'r', encoding='utf-8') as f:
            for line in f:
                # conll line is: id | keyword | lemma | pos | _
                # we want only keyword and lemma
                line = line.split()
                if len(line) > 2 and line[1] not in PUNCTUATION:
                    conllu_lemma_dict[line[1]] = line[2]



1. Error fix: I added above the line:\
<code>if f.endswith(".conllu")</code>\
Without it it gets an error because it tries to read folders and other files as well.\
2. Why do you add the conllu to dictionary?

## Create dictionary from INCEpTION files

In [20]:
from cassis import *
import zipfile
import tempfile

inception_dict = {}
inception_sentences = []

In [21]:
# extract all files in inception folder to temp folder
with tempfile.TemporaryDirectory() as tempdir:
    for f in os.listdir("../assets/NER_assets/INCEpTION_files/"):
        if f.endswith(".zip"):
            with zipfile.ZipFile(os.path.join("../assets/NER_assets/INCEpTION_files/", f), 'r') as zip_ref:
                zip_ref.extractall(tempdir)
    print (tempdir)
    # open typesystem and print content
    with open('{0}/{1}'.format(tempdir, "TypeSystem.xml"), 'rb') as f:
        typesystem = load_typesystem(f)

    # iterate over all files in temp folder
    for f in os.listdir(tempdir):
        # if file is a xmi file
        print (typesystem)
        if f.endswith(".xmi"):
            # load xmi file
            with open(os.path.join(tempdir, f), 'rb') as f:
                # load typesystem from temp folder
                cas = load_cas_from_xmi(f, typesystem=typesystem)
                for token in cas.select('de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Lemma'):
                    inception_dict[token.get_covered_text()] = token.value
                for sentence in cas.select("de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Sentence"):
                    inception_sentences.append(sentence.get_covered_text())


/tmp/tmpw5v_0mg8


In [22]:
# add to big dictionary and sample
big_dict = {**lemma_dict, **dendrosearch_lemma_dict, **conllu_lemma_dict, **inception_dict}

random.sample(list(big_dict.items()), 10)

[('Σαμοθρηίκην', 'Σαμοθρᾴκη'),
 ('νεοκρᾶτα', 'νεοκράς'),
 ('ἐξεφύοντο', 'ἐκφύω'),
 ('κἀγαθὴ', '_'),
 ('ὠτειλήν', 'ὠτειλή'),
 ('χαρέντες', 'χαίρω'),
 ('ληκύθῳ', 'λήκυθος'),
 ('διαφυσηθέντος', '_'),
 ('ἀναπαυσόμενος', 'ἀναπαύω'),
 ('παρεσκεύασμαι', 'παρασκευάζω')]

In [26]:
# create two copies of dictionary, one in NFD, other in NFKC
FIRST_FORMAT = 'NFD'
SECOND_FORMAT = 'NFKC'

big_dict_nfd = {}
big_dict_nfkc = {}

for key, value in big_dict.items():
    big_dict_nfd[normalize(FIRST_FORMAT, key)] = normalize(FIRST_FORMAT, value)
    big_dict_nfkc[normalize(SECOND_FORMAT, key)] = normalize(SECOND_FORMAT, value)

TypeError: normalize() argument 2 must be str, not float

1. Notice the error above: TypeError: normalize() argument 2 must be str, not float
This is probably due to havving numbers as values in dictionary.
2. Why NFD and not NFKD?

In [27]:
# check delta between two dictionaries
delta = {k: big_dict_nfd[k] for k in set(big_dict_nfd) - set(big_dict_nfkc)}
# delta size estimation
print ("delta size:", len(delta))

delta size: 342


# Run NLP pipeline on INCEpTION and Coda files

In [28]:
# merge sentences from inception and coda

sentences = df['Quote'].tolist() + inception_sentences
sentences = list(set(sentences))
random.sample(sentences, 10)

["ὅ τι δ' ἂν διὰ τῆς μήνιγγος ἐς τὸν \r\nἐγκέφαλον ἐσέλθῃ, τοῦτο διαφραδέως ἀκούεται ταύτῃ καὶ μούνη τρήσις \r\nδιὰ τῆς μήνιγγός ἐστι τῆς περὶ τὸν ἐγκέφαλον περιτεταμένης.",
 'Αἱ δ’ ἀρχαὶ τούτων τῶν φλεβῶν, ᾗ σχίζονται τὸ πρῶτον, καλοῦνται σφαγίτιδες',
 ' Τὰ μὲν οὖν περὶ τὸν στόμαχον καὶ τὴν ἀρτηρίαν οὕτως ἔχει, καὶ τὰ περὶ τὴν κοιλίαν',
 'Μεταξὺ δ’ ἔχει τῶν τρήσεων τὴν ἐπιγλωττίδα καλουμένην, ἐπιπτύσσεσθαι δυναμένην ἐπὶ τὸ τῆς ἀρτηρίας τρῆμα τὸ εἰς τὸ στόμα τεῖνον',
 "πρὸς δὲ τὸ γόνυ τὸ ὀστέον τοῦ μηροῦ τοιόνδ' ἐστὶ δίκραιον·",
 'ἔστι γὰρ ὁ μυκτὴρ διχότομος',
 'τὰ γὰρ φλέβια λεπτὰ ἐόντα οὐ δύναται παραδέχεσθαι τὸ φλέγμα ὑπὸ πάχεος καὶ πλήθεος, ἀλλὰ ἀποψύχεται καὶ πήγνυται τὸ αἷμα, καὶ οὕτως ἀποθνήσκει.',
 'Ἅμα δ’ ἡ ἀνάπνευσις καὶ ἔκπνευσις γίνεται εἰς τὸ στῆθος, καὶ ἀδύνατον χωρὶς τοῖς μυκτῆρσιν ἀναπνεῦσαι ἢ ἐκπνεῦσαι, διὰ τὸ ἐκ τοῦ στήθους εἶναι τὴν ἀναπνοὴν

In [30]:
# run nlp pipeline on sentences
# install spacy grc model if not already installed
nlp = spacy.load("grc_proiel_sm") # I use small model for speed but you should use trf (transformer) model for better accuracy

# create DocBin object to store docs
db = DocBin()
# create df to record corrections
corrections_df = pd.DataFrame(columns=['sentence', 'token', 'lemma', 'lemma_corrected', 'dictionary'])
corrected_sentences = 0
for sentence in sentences:
    doc = nlp(sentence)
    # we use our dictionary to replace lemmas which the model didn't recognize correctly
    for token in doc:
        # if token text is a lexical value in our dictionary
        if token.text in big_dict_nfd:
            if token.lemma_ != big_dict_nfd[token.text]:
                corrections_df = corrections_df.append({'sentence': sentence,  'token': token.text, 'lemma': token.lemma_, 'lemma_corrected': big_dict_nfd[token.text], 'dictionary': 'big_dict_nfd'}, ignore_index=True)
                token.lemma_ = big_dict_nfd[token.text]

        elif token.text in big_dict_nfkc:
            if token.lemma_ != big_dict_nfkc[token.text]:
                corrections_df = corrections_df.append({'sentence': sentence,  'token': token.text, 'lemma': token.lemma_, 'lemma_corrected': big_dict_nfkc[token.text], 'dictionary': 'big_dict_nfkc'}, ignore_index=True)
                token.lemma_ = big_dict_nfkc[token.text]
    db.add(doc)


/tmp/ipykernel_506/2939801045.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  corrections_df = corrections_df.append({'sentence': sentence,  'token': token.text, 'lemma': token.lemma_, 'lemma_corrected': big_dict_nfkc[token.text], 'dictionary': 'big_dict_nfkc'}, ignore_index=True)
/tmp/ipykernel_506/2939801045.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  corrections_df = corrections_df.append({'sentence': sentence,  'token': token.text, 'lemma': token.lemma_, 'lemma_corrected': big_dict_nfkc[token.text], 'dictionary': 'big_dict_nfkc'}, ignore_index=True)
/tmp/ipykernel_506/2939801045.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  corrections_df = corrections_df.append({'sentence': sentence,  'token': toke

Again, the deprecation warning 

In [31]:
print ("corrections_df size:", corrections_df.shape)

corrections_df size: (1681, 5)


In [32]:
corrections_df.sample(10)


,sentence,token,lemma,lemma_corrected,dictionary
1280,ᾗ δὲ συνήρτηται κοῖλόν ἐστιν.,συνήρτηται,συνήρτω,συναρτάω,big_dict_nfd
1012,"Ἐν δὲ τοῖς ᾠοτόκοις, οἷον ὄρνισι και...",τοῖς,τοῖς,ὁ,big_dict_nfd
305,Ἡ μὲν οὖν μεγάλη φλὲψ ἐκ τῆς μεγίστης...,δεξιοῖς,δεξιοῖςμι,δεξιός,big_dict_nfd
1450,Ὑπὲρ δὲ τῶν ἐντέρων τὸ μεσεντέριόν ε...,μεσεντέριόν,μεσεντέριόν,μεσεντέριον,big_dict_nfd
308,Ἡ μὲν οὖν μεγάλη φλὲψ ἐκ τῆς μεγίστης...,κοιλίας,κοιλίας,κοιλία,big_dict_nfd
1543,Ἅμα δ’ ἡ ἀνάπνευσις καὶ ἔκπνευσις γίν...,ἐκπνοὴν,ἐκπνοὴν,ἐκπνοή,big_dict_nfd
1288,Τείνει δ’ ἀπὸ μὲν τῆς μεγάλης φλεβὸς ου...,πολλαὶ,πολλαὶ,πολύς,big_dict_nfd
961,Ἔτι προσώπου μέρος τὸ μὲν ὂν τῷ πνευ...,τὸ,τὸ,ὁ,big_dict_nfd
667,Ἔστι δ’ ἡ ἀορτὴ ἀπὸ μὲν τῆς καρδίας ...,καρδίας,καρδίῖς,καρδία,big_dict_nfd
1393,Μεταξὺ δ’ ἔχει τῶν τρήσεων τὴν ἐπιγλωττ...,Μεταξὺ,Μεταξὺ,μεταξύ,big_dict_nfd


In [33]:
# find how many corrected by each dictionary
corrections_df.groupby('dictionary').count()


,sentence,token,lemma,lemma_corrected
dictionary,,,,
big_dict_nfd,1543,1543,1543,1543
big_dict_nfkc,138,138,138,138


In [34]:
corrections_df[corrections_df['dictionary'] == 'big_dict_nfkc'].sample(10)

,sentence,token,lemma,lemma_corrected,dictionary
1001,τῷ δὲ \r\nδικραίῳ τούτῳ τὸ ὀστέον ἡ κνήμη καλε...,δὲ,δέ,δὲ,big_dict_nfkc
1376,"αἱ δὲ περόναι ταύτη \r\nἁπαλαὶ ἐοῦσαι, αἱ μὲν ...",δὲ,δέ,δὲ,big_dict_nfkc
1637,Πρὸς δὲ τὸν καυλὸν τὸν τῆς κύστεως \r\nσυνήρ...,δὲ,δέ,δὲ,big_dict_nfkc
383,"ἡ δὲ διὰ μέσου ἐπικίνδυνος αὕτη, καὶ ὅταν ῥαγῇ...",δὲ,δέ,δὲ,big_dict_nfkc
532,Τὸν αὐτὸν δὲ τρόπον καὶ ἐν τῷ θήλει πάντα πέφυ...,δὲ,δέ,δὲ,big_dict_nfkc
1138,"μοῦναι γὰρ αὗται οὐκ ἄρδουσι τῶν φλεβῶν, ἀλλ᾽ ...",φλεβῶν,φλεβής,φλέψ,big_dict_nfkc
627,"παρὰ δὲ τὸ ὀστέον περόναι δύο παρήκουσιν, ἡ μὲ...",κάτω,κάτω,κάτω,big_dict_nfkc
229,"ὅσα γὰρ τὰ ὀστέα, τοσαῦτα καὶ τὰ ἄρθρα, καὶ ἐν...",δὲ,δέ,δὲ,big_dict_nfkc
998,καὶ ἡ μὲν ἔνθα ἡ δὲ ἔνθα τῶν \r\nσπονδύλων ἐλθ...,δὲ,δέ,δὲ,big_dict_nfkc
530,"ἡ δὲ κοίλη φλέψ περαίνεται μὲν ὡς ὁ οἰσοφάγος,...",δὲ,δέ,δὲ,big_dict_nfkc


In [35]:
corrections_df[corrections_df['dictionary'] == 'big_dict_nfd'].sample(10)

,sentence,token,lemma,lemma_corrected,dictionary
417,Φυσωμένης δὲ τῆς ἀρτηρίας διαδίδωσιν εἰ...,Φυσωμένης,Φυσωμένη,εἰς,big_dict_nfd
970,Καὶ ὁ μὲν πλεύμων οὐκ ἐν αὑτῷ ἀλλ’ ε...,μὲν,μὲν,μέν,big_dict_nfd
284,Ἅμα δ’ ἡ ἀνάπνευσις καὶ ἔκπνευσις γίν...,κεφαλῆς,κεφαλῆς,κεφαλή,big_dict_nfd
1614,μετὰ γὰρ τὴν κοιλίαν ἡ τῶν ἐντέρων ἔ...,τοῖς,τοῖς,ὁ,big_dict_nfd
222,Ἐξήρτηται δ' ἐκ τῆς μεγάλης φλεβὸς καὶ...,πολλαὶ,πολλαὶ,πολύς,big_dict_nfd
907,Ἅμα δ’ ἡ ἀνάπνευσις καὶ ἔκπνευσις γίν...,ἐκπνεῦσαι,ἐκπνεῦζω,ἐκπνέω,big_dict_nfd
374,Ἕτερον δὲ μέρος ἀπὸ τῶν ἀριστερῶν τῆ...,μεγάλης,μεγάλης,μέγας,big_dict_nfd
425,Ἔστι δ’ ἡ ἀορτὴ ἀπὸ μὲν τῆς καρδίας ...,καρδίας,καρδίῖς,καρδία,big_dict_nfd
450,τὸ δ’ εἰς τὸ κάτω τῆς καρδίας μέρος τῆ...,μεγάλης,μεγάλης,μέγας,big_dict_nfd
721,Ταύτῃ δὲ τὸ πέρας συνήρτηται τῆς γλώττης.,γλώττης,γλώττη,γλῶσσα,big_dict_nfd


In [36]:
# save to spacy file
db.to_disk("../assets/NER_assets/corrected_sentences.spacy")


The spacy dataset should be exported to '../corpus/' folder.\
More specifically:\
train to '..corpus/train/lemma_train/'\
dev to '../corpus/dev/lemma_dev/'\
test to '../corpus/test/lemma_test/